In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(17)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.14:54990 Dashboard: http://10.148.1.14:8787/status,Cluster Workers: 30 Cores: 120 Memory: 1500.00 GB


_____
# For each face

In [22]:
from mitequinox.binary import *

V = ['SSU','SSV']

out_dir = '/home1/scratch/xyu/mit_zarr/UV/LLC4320/'  
root_data_dir = '/home/datawork-lops-osi/data/mit4320/'
grid_dir = root_data_dir+'grid/'

face_all = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#face_all = [1,2,4,5,7,8,10,11]
#face_all = [7, 8, 9, 10, 11, 12]

sli = slice(0,None,1) # required to make worldwide plots, otherwise, half the globe is plotted
it = np.arange(6).astype(int)*Nt # tmp


In [ ]:
V = ['SSU']
Nt = 30*24*2 # time windows to consider

for face in face_all :  
    
    for v in V:

        for i, t in enumerate(it):

            data_dir = root_data_dir+v+'/'
              
            iters, time = get_iters_time(v, data_dir, delta_t=25.)
    
            p = 'C'
            vdrop = ['XC','YC','CS','SN','rA','Depth']
            if v is 'SSU':
                p = 'W'
                vdrop = ['dxC','dyG','rAw']
            
            elif v is 'SSV':
                p = 'S'
                vdrop = ['dxG','dyC','rAs']
            
            ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        
            dv = ds[v].isel(time=slice(t, t+Nt), face=face, i_g=sli, j=sli) 
            dv = dv.drop(vdrop).to_dataset()        
            file_out = out_dir+'/%s_f%02d_t%02d.zarr'%(v,face,i)

            dv.to_zarr(file_out, mode='w')                    
            dsize = getsize(file_out)
            print('face=%d / i=%d done'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


face=0 / i=0 done
face=0 / i=1 done
face=0 / i=2 done


In [24]:
V = ['SSV']
Nt = 30*24*2 # time windows to consider

for face in face_all :  
    
    for v in V:

        for i, t in enumerate(it):

            data_dir = root_data_dir+v+'/'
            iters, time = get_iters_time(v, data_dir, delta_t=25.)
    
            p = 'C'
            vdrop = ['XC','YC','CS','SN','rA','Depth']
            if v is 'SSU':
                p = 'W'
                vdrop = ['dxC','dyG','rAw']
            
            elif v is 'SSV':
                p = 'S'
                vdrop = ['dxG','dyC','rAs']
            
            ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        
            dv = ds[v].isel(time=slice(t, t+Nt), face=face, i=sli, j_g=sli) 
            dv = dv.drop(vdrop).to_dataset()        
            file_out = out_dir+'/%s_f%02d_t%02d.zarr'%(v,face,i)

            dv.to_zarr(file_out, mode='w')                    
            dsize = getsize(file_out)
            print('face=%d / i=%d done'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


face=0 / i=0 done
face=0 / i=1 done
face=0 / i=2 done
face=0 / i=3 done
face=0 / i=4 done
face=0 / i=5 done
face=1 / i=0 done
face=1 / i=1 done
face=1 / i=2 done
face=1 / i=3 done
face=1 / i=4 done
face=1 / i=5 done
face=2 / i=0 done
face=2 / i=1 done
face=2 / i=2 done
face=2 / i=3 done
face=2 / i=4 done
face=2 / i=5 done
face=3 / i=0 done
face=3 / i=1 done
face=3 / i=2 done
face=3 / i=3 done
face=3 / i=4 done
face=3 / i=5 done
face=4 / i=0 done
face=4 / i=1 done
face=4 / i=2 done
face=4 / i=3 done
face=4 / i=4 done
face=4 / i=5 done
face=5 / i=0 done
face=5 / i=1 done
face=5 / i=2 done
face=5 / i=3 done
face=5 / i=4 done
face=5 / i=5 done
face=6 / i=0 done
face=6 / i=1 done
face=6 / i=2 done
face=6 / i=3 done
face=6 / i=4 done
face=6 / i=5 done
face=7 / i=0 done
face=7 / i=1 done
face=7 / i=2 done
face=7 / i=3 done
face=7 / i=4 done
face=7 / i=5 done
face=8 / i=0 done
face=8 / i=1 done
face=8 / i=2 done
face=8 / i=3 done
face=8 / i=4 done
face=8 / i=5 done
face=9 / i=0 done
face=9 / i

In [26]:
cluster.scheduler.close()

<Future finished result=None>